### 1) Обучение модели LSTM

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as T
import torchvision.models.video as models
from torchvision.models.video import R3D_18_Weights
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter

# Config
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_FRAMES = 16
IMG_SIZE = 112
BATCH_SIZE = 16
EPOCHS = 20
PATIENCE = 3

In [ ]:
!mkdir -p /content/data

In [ ]:
#!unzip -q /content/other.zip -d /content/data
!unzip -q /content/Captures.zip -d /content/data

In [ ]:
#!unzip -q /content/drive/MyDrive/archive.zip -d /content/data

# или
!pip install gdown
FILE_ID = '1TVhCj9cr5WMEmsyxGxUahuAcUXXXeQ7d'
!gdown --id $FILE_ID
!unzip -q /content/archive.zip -d /content/data

In [ ]:
BASE_PATH = '/content/data'

def get_all_videos(base_path):
    videos = []
    labels = []
    for label in os.listdir(base_path):
        folder = os.path.join(base_path, label)
        if os.path.isdir(folder):
            for file in os.listdir(folder):
                if file.endswith(".mp4"):
                    videos.append(os.path.join(folder, file))
                    labels.append(label)
    return videos, labels

videos, labels = get_all_videos(BASE_PATH)
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)

In [ ]:
for i, class_name in enumerate(le.classes_):
    print(f"Класс {i}: {class_name}")

import json
class_mapping = {i: class_name for i, class_name in enumerate(le.classes_)}
with open('class_mapping.json', 'w') as f:
    json.dump(class_mapping, f)

In [ ]:
print("\nСтруктура объединенного датасета:")
for class_name in sorted(os.listdir(BASE_PATH)):
    class_path = os.path.join(BASE_PATH, class_name)
    if os.path.isdir(class_path):
        video_count = len([f for f in os.listdir(class_path)
                          if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))])
        print(f"  {class_name}: {video_count} видео")

In [ ]:
class_counts = Counter(labels)
plt.figure(figsize=(15, 15))
shown_labels = set()
shown = 0
for i, path in enumerate(videos):
    label = labels[i]
    if label in shown_labels:
        continue
    cap = cv2.VideoCapture(path)
    success, frame = cap.read()
    cap.release()
    if success:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.subplot(5, 5, shown + 1)
        plt.imshow(frame)
        plt.title(label)
        plt.axis("off")
        shown_labels.add(label)
        shown += 1
    if shown >= len(class_counts):
        break
plt.tight_layout()
plt.show()

In [ ]:
!pip install ultralytics

In [ ]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Загружаем модель для позы
pose_model = YOLO('yolo11s-pose.pt')

def extract_pose_sequence(video_path, max_frames=60):
    """Извлекает последовательность поз из видео"""
    cap = cv2.VideoCapture(video_path)
    poses = []
    frame_count = 0

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # Детектируем позу
        results = pose_model(frame, verbose=False)

        if len(results[0].keypoints) > 0:
            # Берем позу первого человека
            keypoints = results[0].keypoints[0].data.cpu().numpy()[0]  # [17, 3]
            poses.append(keypoints.flatten())
        else:
            # Если человека нет - заполняем нулями
            poses.append(np.zeros(51))

        frame_count += 1

    cap.release()
    return np.array(poses)

# Извлекаем последовательности для всех видео
print("Извлекаем позы из видео...")
video_sequences = []

for i, video_path in enumerate(videos):
    if i % 10 == 0:
        print(f"Обработано {i}/{len(videos)} видео")

    sequence = extract_pose_sequence(video_path)
    video_sequences.append(sequence)

print(f"Извлечено {len(video_sequences)} последовательностей")

In [ ]:
lengths = [len(seq) for seq in video_sequences]

print("Статистика длин последовательностей:")
print(f"Min: {np.min(lengths)}")
print(f"Max: {np.max(lengths)}")
print(f"Mean: {np.mean(lengths):.1f}")
print(f"Median (50%): {np.median(lengths)}")
print(f"75% перцентиль: {np.percentile(lengths, 75)}")
print(f"90% перцентиль: {np.percentile(lengths, 90)}")

In [ ]:
class PoseDatasetWithAugmentation(Dataset):
    def __init__(self, sequences, labels, seq_length=50, is_training=True, augment_prob=0.75):
        self.sequences = sequences
        self.labels = labels
        self.seq_length = seq_length
        self.is_training = is_training
        self.augment_prob = augment_prob

    def random_drop_frames(self, sequence, target_length):
        """
        Удаляет случайные кадры из последовательности (имитация потерь)
        sequence: [N, 51] - исходная последовательность
        target_length: нужная длина
        """
        if len(sequence) <= target_length:
            return sequence

        # Случайно выбираем какие кадры сохранить
        indices_to_keep = np.random.choice(
            len(sequence),
            size=target_length,
            replace=False
        )
        indices_to_keep.sort()

        return sequence[indices_to_keep]

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx].copy()  # [N, 51]
        label = self.labels[idx]

        # Нормализуем позу
        sequence = self.normalize_pose(sequence)

        # Аугментации только в режиме обучения
        if self.is_training and np.random.random() < self.augment_prob:
            sequence = self.augment_sequence(sequence)

        # Приводим к фиксированной длине
        processed_seq = self.pad_or_truncate(sequence)

        return torch.FloatTensor(processed_seq), torch.tensor(label)

    def normalize_pose(self, sequence):
        """Нормализует координаты относительно таза"""
        if len(sequence) == 0:
            return sequence

        seq_reshaped = sequence.reshape(-1, 17, 3)

        for i in range(len(seq_reshaped)):
            frame = seq_reshaped[i]

            # Находим центр таза (точки 11, 12)
            if np.all(frame[11, 2] > 0.1) and np.all(frame[12, 2] > 0.1):
                pelvis_center = (frame[11, :2] + frame[12, :2]) / 2
            else:
                # Используем среднее всех видимых точек
                visible_points = frame[frame[:, 2] > 0.1]
                if len(visible_points) > 0:
                    pelvis_center = np.mean(visible_points[:, :2], axis=0)
                else:
                    continue

            # Центрируем все точки
            seq_reshaped[i, :, :2] = seq_reshaped[i, :, :2] - pelvis_center

        return seq_reshaped.reshape(-1, 51)

    def augment_sequence(self, sequence):
        """Применяет аугментации к последовательности поз"""
        if len(sequence) == 0:
            return sequence

        seq_reshaped = sequence.reshape(-1, 17, 3)

        # 1. Зеркалирование (горизонтальное отражение)
        if np.random.random() < 0.5:
            seq_reshaped = self.mirror_sequence(seq_reshaped)

        # 2. Небольшой поворот (2-5 градусов)
        if np.random.random() < 0.5:
            angle = np.random.uniform(-10, 10)  # градусы
            seq_reshaped = self.rotate_sequence(seq_reshaped, angle)

        # 3. Случайный сдвиг точек
        if np.random.random() < 0.5:
            noise_std = np.random.uniform(0.05, 0.09)
            seq_reshaped = self.add_jitter(seq_reshaped, noise_std)

        # 4. Случайное масштабирование
        if np.random.random() < 0.25:
            scale = np.random.uniform(0.9, 1.1)
            seq_reshaped[:, :, :2] = seq_reshaped[:, :, :2] * scale

        return seq_reshaped.reshape(-1, 51)

    def mirror_sequence(self, sequence):
        """Зеркалирует позу (горизонтальное отражение)"""
        mirrored = sequence.copy()

        # Инвертируем x координаты
        mirrored[:, :, 0] = -mirrored[:, :, 0]

        # Меняем местами левые и правые суставы
        left_right_pairs = [(1, 2), (3, 4), (5, 6), (7, 8), (9, 10), (11, 12), (13, 14), (15, 16)]

        for left_idx, right_idx in left_right_pairs:
            temp = mirrored[:, left_idx, :].copy()
            mirrored[:, left_idx, :] = mirrored[:, right_idx, :]
            mirrored[:, right_idx, :] = temp

        return mirrored

    def rotate_sequence(self, sequence, angle_degrees):
        """Поворачивает позу на небольшой угол"""
        angle_rad = np.radians(angle_degrees)
        rotation_matrix = np.array([
            [np.cos(angle_rad), -np.sin(angle_rad)],
            [np.sin(angle_rad), np.cos(angle_rad)]
        ])

        rotated = sequence.copy()
        rotated[:, :, :2] = np.dot(rotated[:, :, :2], rotation_matrix.T)

        return rotated

    def add_jitter(self, sequence, noise_std):
        """Добавляет случайный шум к координатам"""
        jittered = sequence.copy()
        noise = np.random.normal(0, noise_std, jittered[:, :, :2].shape)
        jittered[:, :, :2] += noise
        return jittered

    def pad_or_truncate(self, sequence):
        """Новая версия - удаляем случайные кадры вместо обрезки концов"""
        if len(sequence) > self.seq_length:
            # Вместо обрезки конца - удаляем случайные кадры
            sequence = self.random_drop_frames(sequence, self.seq_length)
        else:
            # Дополняем нулями как раньше
            padded = np.zeros((self.seq_length, 51))
            padded[:len(sequence)] = sequence
            sequence = padded

        return sequence

In [ ]:
def prepare_lstm_data(video_sequences, labels, test_size=0.20):
    """Подготавливает данные для LSTM"""
    # Разделяем на train/test
    X_train, X_test, y_train, y_test = train_test_split(
        video_sequences, labels,
        test_size=test_size, random_state=42, stratify=labels
    )

    # Создаем датасеты
    train_dataset = PoseDatasetWithAugmentation(X_train, y_train, seq_length=50, is_training=True)
    test_dataset = PoseDatasetWithAugmentation(X_test, y_test, seq_length=50, is_training=False)

    # Создаем DataLoader'ы
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    return train_loader, test_loader, len(np.unique(labels))

# Подготавливаем данные
train_loader, test_loader, num_classes = prepare_lstm_data(video_sequences, encoded_labels)
print(f"Количество классов: {num_classes}")

In [ ]:
import torch.nn as nn
class EnhancedPoseLSTM(nn.Module):
    def __init__(self, input_size=51, hidden_size=256, num_layers=3, num_classes=23, dropout=0.4):
        super(EnhancedPoseLSTM, self).__init__()

        self.num_layers = num_layers
        self.hidden_size = hidden_size

        # Более глубокая LSTM архитектура
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True
        )

        # Улучшенный attention механизм
        self.attention = nn.Sequential(
            nn.Linear(hidden_size * 2, 128),
            nn.Tanh(),
            nn.Dropout(dropout),
            nn.Linear(128, 32),
            nn.Tanh(),
            nn.Linear(32, 1)
        )

        # Более глубокая классифицирующая сеть
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_size * 2, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),

            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),

            nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),

            nn.Dropout(dropout),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        # LSTM forward
        lstm_out, (hidden, cell) = self.lstm(x)  # [batch_size, seq_len, hidden_size*2]

        # Multi-head attention (улучшенная версия)
        attention_weights = torch.softmax(
            self.attention(lstm_out).squeeze(-1), dim=1
        )

        # Взвешенная сумма с attention
        context_vector = torch.bmm(
            attention_weights.unsqueeze(1), lstm_out
        ).squeeze(1)

        # Классификация
        output = self.classifier(context_vector)
        return output

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EnhancedPoseLSTM(num_classes=num_classes, dropout=0.4).to(device)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def train_model_with_early_stopping(model, train_loader, test_loader, num_epochs=100, patience=15):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=7, factor=0.5)

    best_accuracy = 0
    patience_counter = 0
    train_losses = []
    test_accuracies = []

    for epoch in range(num_epochs):
        # Обучение
        model.train()
        running_loss = 0.0

        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()

            # Gradient clipping для стабильности
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            running_loss += loss.item()

        # Валидация
        model.eval()
        test_correct = 0
        test_total = 0

        with torch.no_grad():
            for data, targets in test_loader:
                data, targets = data.to(device), targets.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                test_total += targets.size(0)
                test_correct += (predicted == targets).sum().item()

        test_accuracy = 100 * test_correct / test_total
        test_accuracies.append(test_accuracy)
        train_losses.append(running_loss / len(train_loader))

        scheduler.step(test_accuracy)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Loss: {running_loss/len(train_loader):.4f}, '
              f'Test Acc: {test_accuracy:.2f}%')

        # Early stopping
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            patience_counter = 0
            torch.save(model.state_dict(), 'best_pose_lstm.pth')
            print(f"Новый лучший результат! Сохранена модель с точностью {best_accuracy:.2f}%")
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping на эпохе {epoch+1}")
            break

    # Загружаем лучшую модель
    model.load_state_dict(torch.load('best_pose_lstm.pth'))
    return train_losses, test_accuracies, best_accuracy

In [ ]:
# Запускаем обучение
print("Начинаем обучение LSTM с аугментациями...")
train_losses, test_accuracies, best_acc = train_model_with_early_stopping(
    model, train_loader, test_loader, num_epochs=150, patience=20)

print(f"Лучшая точность: {best_acc:.2f}%")

### Взаимодействие с готовой моделью (ПЕРЕВЕДЕНО В ПАЙПЛАЙН, ВЫПОЛНЯТЬ НЕ НАДО)


In [ ]:
#ФУНКЦИЯ ДЕТЕКТИТ И ТРЕКАЕТ ЛЮДЕЙ
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
def process_video_for_lstm(video_path, min_visibility_seconds=5, min_detection_percent=40):
    """
    Трекинг людей на видео с фильтрацией

    Возвращает: {track_id: {"keypoints_original": [...], ...}}
    """
    # Загрузка модели
    model = YOLO('yolo11n-pose.pt')
    cap = cv2.VideoCapture(video_path)

    tracker_config = {
      'tracker_type': 'bytetrack',
      'track_high_thresh': 0.6,
      'track_low_thresh': 0.3,
      'new_track_thresh': 0.7,
      'track_buffer': 10,
      'match_thresh': 0.70,
      'fuse_score': True}

    import tempfile
    import yaml
    with tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False) as f:
      yaml.dump(tracker_config, f)
      tracker_file = f.name

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        fps = 30

    # Хранилище данных
    tracks = defaultdict(lambda: {
        "keypoints_original": [],  # ТОЛЬКО оригинальные координаты
        "timestamps": [],
        "frame_indices": [],
        "confidences": [],
        "bboxes": []
    })

    # Основной цикл трекинга
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Детекция с трекингом
        results = model.track(frame, persist=True, verbose=False, conf=0.5, iou=0.4, tracker=tracker_file)

        # Обработка детекций
        if (results[0].boxes is not None and
            results[0].boxes.id is not None and
            results[0].keypoints is not None):

            track_ids = results[0].boxes.id.cpu().numpy().astype(int)
            keypoints_list = results[0].keypoints.data.cpu().numpy()

            for i, track_id in enumerate(track_ids):
                kps = keypoints_list[i]
                if len(kps.shape) == 3:
                    kps = kps[0]  # [17, 3]

                # Сохраняем ТОЛЬКО оригинальные координаты
                tracks[track_id]["keypoints_original"].append(kps.flatten())
                tracks[track_id]["timestamps"].append(frame_idx / fps)
                tracks[track_id]["frame_indices"].append(frame_idx)

        frame_idx += 1

    cap.release()

    # ФИЛЬТРАЦИЯ
    filtered_tracks = {}

    for track_id, data in tracks.items():
        if len(data["timestamps"]) == 0:
            continue

        # Вычисляем метрики
        start_time = data["timestamps"][0]
        end_time = data["timestamps"][-1]
        duration = end_time - start_time

        # Вычисляем процент кадров с детекцией
        detection_windows = []
        window_size = 1.0  # 1 секунда

        current_window_start = start_time
        while current_window_start < end_time:
            window_end = current_window_start + window_size

            # Кадры в этом окне
            frames_in_window = [
                i for i, t in enumerate(data["timestamps"])
                if current_window_start <= t < window_end
            ]

            if len(frames_in_window) > 0:
                detection_windows.append(len(frames_in_window))

            current_window_start = window_end

        # Если были окна с детекцией
        if detection_windows:
            avg_detection_in_window = np.mean(detection_windows)
            expected_frames_per_window = window_size * fps
            detection_percentage = (avg_detection_in_window / expected_frames_per_window) * 100
        else:
            detection_percentage = 0

        # Критерии фильтрации
        condition1 = (duration >= min_visibility_seconds and
                     detection_percentage >= min_detection_percent)
        condition2 = (detection_percentage >= 80)

        if condition1 or condition2:
            # Конвертируем в numpy
            filtered_tracks[track_id] = {
                "keypoints_original": np.array(data["keypoints_original"]),  # Только оригинальные
                "timestamps": np.array(data["timestamps"]),
                "frame_indices": np.array(data["frame_indices"]),
                "duration": duration,
                "detection_percentage": detection_percentage
            }
    return filtered_tracks

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
class PosePreprocessor:
    """Класс для предобработки последовательностей ключевых точек"""
    def __init__(self, seq_length=50, training=False):
        self.seq_length = seq_length
        self.training = training

    def random_drop_frames(self, sequence, target_length):
        """
        Удаляет случайные кадры из последовательности (имитация потерь)
        sequence: [N, 51] - исходная последовательность
        target_length: нужная длина
        """
        if len(sequence) <= target_length:
            return sequence

        # Случайно выбираем какие кадры сохранить
        indices_to_keep = np.random.choice(
            len(sequence),
            size=target_length,
            replace=False
        )
        indices_to_keep.sort()

        return sequence[indices_to_keep]

    def normalize_pose(self, sequence):
        """Нормализует координаты относительно таза"""
        if len(sequence) == 0:
            return sequence

        seq_reshaped = sequence.reshape(-1, 17, 3)

        for i in range(len(seq_reshaped)):
            frame = seq_reshaped[i]

            # Находим центр таза (точки 11, 12)
            if np.all(frame[11, 2] > 0.1) and np.all(frame[12, 2] > 0.1):
                #print('awesome norm')
                pelvis_center = (frame[11, :2] + frame[12, :2]) / 2
            else:
                # Используем среднее всех видимых точек
                visible_points = frame[frame[:, 2] > 0.1]
                if len(visible_points) > 0:
                    #print('meh norm')
                    pelvis_center = np.mean(visible_points[:, :2], axis=0)
                else:
                    #print('no norm')
                    continue

            # Центрируем все точки
            seq_reshaped[i, :, :2] = seq_reshaped[i, :, :2] - pelvis_center

        return seq_reshaped.reshape(-1, 51)

    def pad_or_truncate(self, sequence):
        """Приводит последовательность к фиксированной длине self.seq_length"""
        if len(sequence) > self.seq_length:
            # Вместо обрезки конца - удаляем случайные кадры
            sequence = self.random_drop_frames(sequence, self.seq_length)
        else:
            # Дополняем нулями
            padded = np.zeros((self.seq_length, 51))
            padded[:len(sequence)] = sequence
            sequence = padded

        return sequence

    def preprocess(self, keypoints_sequence):
        """
        Полный пайплайн предобработки для одного сегмента
        keypoints_sequence: [n_frames, 51]
        возвращает: [seq_length, 51]
        """
        keypoints_sequence_copy = keypoints_sequence.copy()
        # 1. Нормализация
        normalized = self.normalize_pose(keypoints_sequence_copy)

        # 2. Приведение к фиксированной длине (60 кадров)
        processed = self.pad_or_truncate(normalized)

        return processed


def load_lstm_model(model_path, class_names, device='cpu'):
    """
    Загружает LSTM модель с весами
    """
    print(f"Загрузка модели из {model_path}...")

    model = EnhancedPoseLSTM(
        input_size=51,
        hidden_size=256,
        num_layers=3,
        num_classes=23,
        dropout=0.4
    )

    # Загружаем веса
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    print(f"Модель загружена на {device}")
    return model

def classify_person_sequence(segment, lstm_model, preprocessor, class_names, device='cpu'):
    """
    Реальная классификация с вашей LSTM моделью
    """
    keypoints = segment["keypoints"]  # [n_frames, 51] - ОРИГИНАЛЬНЫЕ координаты

    if len(keypoints) == 0:
        return {
            "predicted_class": "no_exercise",
            "confidence": 0.0,
            "class_idx": -1
        }

    # ВАЖНО: preprocessor.preprocess() нормализует координаты внутри себя
    # Он вызывает normalize_pose() → вычитает pelvis_center
    processed = preprocessor.preprocess(keypoints)  # [50, 51] - НОРМАЛИЗОВАННЫЕ координаты

    debug = False
    if debug:
        print(f"До нормализации: X [{np.min(keypoints[:, 0::3]):.1f}, {np.max(keypoints[:, 0::3]):.1f}], Y [{np.min(keypoints[:, 1::3]):.1f}, {np.max(keypoints[:, 1::3]):.1f}]")
        print(f"После нормализации: X [{np.min(processed[:, 0::3]):.1f}, {np.max(processed[:, 0::3]):.1f}], Y [{np.min(processed[:, 1::3]):.1f}, {np.max(processed[:, 1::3]):.1f}]")

    # Подготовка тензора
    input_tensor = torch.tensor(processed, dtype=torch.float32)
    input_tensor = input_tensor.unsqueeze(0).to(device)  # [1, 50, 51]

    # Предсказание
    with torch.no_grad():
        outputs = lstm_model(input_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        confidence, predicted_idx = torch.max(probabilities, dim=1)

    return {
        "predicted_class": class_names[predicted_idx.item()],
        "confidence": confidence.item(),
        "class_idx": predicted_idx.item()
    }

In [ ]:
# ОСНОВАНЯ ФУНКЦИЯ, КОТОРАЯ ИДЕТ ПО ВИДЕО С НУЖНЫМ ОКНОМ
def full_process(video_path, model_path, min_visibility_sec=5, min_detection_percent=40):
    """
    Исправленная версия full_process
    """
    # Конфигурация
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Загружаем LSTM модель
    lstm_model = load_lstm_model(model_path, [], device)
    preprocessor = PosePreprocessor(seq_length=50, training=False)

    # 1. Трекинг - получаем только оригинальные координаты
    print(f"Трекинг видео: {video_path}")
    tracks = process_video_for_lstm(
        video_path=video_path,
        min_visibility_seconds=min_visibility_sec,
        min_detection_percent=min_detection_percent
    )
    clean_tracks = tracks
    if not tracks:
        print("Нет людей для анализа")
        return {}, {}
    # 2. Для каждого человека разбиваем на 2-секундные сегменты
    predictions = {}

    for track_id, track_data in tracks.items():
        print(f"\nОбработка человека ID {track_id}")
        # Используем ОРИГИНАЛЬНЫЕ координаты
        keypoints_original = track_data["keypoints_original"]
        timestamps = track_data["timestamps"]

        if len(timestamps) < 2:
            continue

        # Получаем FPS видео
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0:
            fps = 30
        cap.release()

        window_frames = 50#int(2 * fps)  # 2 секунды
        step_frames = 50#int(1 * fps)    # 1 секунда перекрытия

        segments = []

        for start in range(0, len(keypoints_original) - window_frames + 1, step_frames):
            end = start + window_frames

            segment = {
                "track_id": track_id,
                "start_time": float(timestamps[start]),
                "end_time": float(timestamps[end-1]),
                "keypoints": keypoints_original[start:end],  # Оригинальные координаты
                "segment_idx": len(segments)
            }
            segments.append(segment)

        print(f"  Сегментов: {len(segments)}")

        # 3. Классификация каждого сегмента|
        segment_predictions = []
        import json
        with open('class_mapping.json', 'r') as f:
          class_mapping = json.load(f)
        class_names = [class_mapping[str(i)] for i in range(len(class_mapping))]
        for segment in segments:
            prediction = classify_person_sequence(
                segment=segment,
                lstm_model=lstm_model,
                preprocessor=preprocessor,
                class_names=class_names,
                device=device
            )

            result = {
                "track_id": track_id,
                "segment_idx": segment["segment_idx"],
                "start_time": segment["start_time"],
                "end_time": segment["end_time"],
                "predicted_class": prediction["predicted_class"],
                "confidence": prediction["confidence"],
                "class_idx": prediction["class_idx"]
            }
            segment_predictions.append(result)

        predictions[track_id] = segment_predictions
    return clean_tracks, predictions

In [ ]:
tracks, predictions = full_process(
      video_path="/content/test_final.mp4",
      model_path="/content/best_pose_lstm_new.pth",
      min_visibility_sec=2,
      min_detection_percent=60
  )

Загрузка модели из /content/best_pose_lstm_new.pth...
Модель загружена на cuda
Трекинг видео: /content/test_final.mp4

Обработка человека ID 1
  Сегментов: 20

Обработка человека ID 5
  Сегментов: 6

Обработка человека ID 7
  Сегментов: 0

Обработка человека ID 9
  Сегментов: 1

Обработка человека ID 20
  Сегментов: 0

Обработка человека ID 21
  Сегментов: 1


### Расчет статистик (ВАЖНАЯ ШТУКА)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import os

def calculate_track_statistics(tracks, predictions):
    """
    Рассчитывает статистику по каждому треку с фильтрацией по количеству детекций

    Параметры:
    -----------
    tracks : dict
        Словарь с треками от функции process_video_for_lstm
    predictions : dict
        Словарь с предсказаниями от функции full_process

    Возвращает:
    -----------
    dict : отфильтрованная статистика по каждому треку
    DataFrame : таблица со статистикой
    """

    statistics = {}
    filtered_tracks = {}

    # Фильтрация треков по количеству детекций (сегментов)
    print(f"🔍 Фильтрация треков: минимум 3 детекции (сегмента)")
    original_count = len(tracks)

    for track_id in tracks.keys():
        # Количество сегментов для этого трека
        track_segments = predictions.get(track_id, [])
        segments_count = len(track_segments)

        if segments_count > 2:  # Больше 2 детекций
            filtered_tracks[track_id] = tracks[track_id]
            print(f"  Сохранен трек {track_id}: {segments_count} сегментов")
        else:
            print(f"  Пропущен трек {track_id}: только {segments_count} сегментов")

    print(f"📊 После фильтрации треков: {len(filtered_tracks)} из {original_count} треков")

    # Расчет статистики по отфильтрованным трекам
    for track_id, track_data in filtered_tracks.items():
        # 1. Суммарное время видимости (из tracks)
        total_time_visible = track_data.get("duration", 0)

        # Получаем timestamps для расчетов
        timestamps = track_data.get("timestamps", None)

        if timestamps is not None and len(timestamps) > 0:
            start_time = float(timestamps[0])
            end_time = float(timestamps[-1])
        else:
            start_time = 0
            end_time = 0

        # 2. Собираем информацию об упражнениях
        exercises_info = defaultdict(lambda: {
            "total_time": 0,
            "segments": 0,
            "segment_details": []
        })

        if track_id in predictions and predictions[track_id]:
            for segment in predictions[track_id]:
                exercise = segment["predicted_class"]
                segment_duration = segment["end_time"] - segment["start_time"]

                # Пропускаем "no_exercise"
                if exercise == "no_exercise":
                    continue

                # Сохраняем детали сегмента
                segment_info = {
                    "start_time": segment["start_time"],
                    "end_time": segment["end_time"],
                    "duration": segment_duration,
                    "confidence": segment.get("confidence", 0),
                    "segment_idx": segment.get("segment_idx", 0)
                }
                exercises_info[exercise]["segment_details"].append(segment_info)

        # 3. Фильтрация упражнений: минимум 3 детекции для учета
        filtered_exercises_info = {}
        filtered_out_exercises = []

        for exercise, info in exercises_info.items():
            segments_count = len(info["segment_details"])

            if segments_count > 2:  # Больше 2 детекций
                # Суммируем общее время упражнения
                total_time = sum(seg["duration"] for seg in info["segment_details"])
                filtered_exercises_info[exercise] = {
                    "total_time": total_time,
                    "segments": segments_count
                }
                print(f"  Трек {track_id}: упражнение '{exercise}' - {segments_count} сегментов")
            else:
                filtered_out_exercises.append(exercise)

        if filtered_out_exercises:
            print(f"  Трек {track_id}: пропущены упражнения {filtered_out_exercises} (≤2 сегментов)")

        # 4. Количество разнообразных упражнений (исключая "other")
        unique_exercises = set()
        for exercise in filtered_exercises_info.keys():
            if exercise != "other" and exercise != "no_exercise":
                unique_exercises.add(exercise)
        num_unique_exercises = len(unique_exercises)

        # 5. Время выполнения каждого упражнения относительно суммарного времени
        exercise_percentages = {}
        for exercise, info in filtered_exercises_info.items():
            if exercise != "other" and exercise != "no_exercise":
                percentage = (info["total_time"] / total_time_visible * 100) if total_time_visible > 0 else 0
                exercise_percentages[exercise] = {
                    "time_seconds": round(info["total_time"], 2),
                    "percentage": round(percentage, 1),
                    "segments": info["segments"]
                }

        # 6. Время без упражнений "other" и "no_exercise"
        total_time_without_other = total_time_visible
        if "other" in filtered_exercises_info:
            total_time_without_other -= filtered_exercises_info["other"]["total_time"]

        # Собираем всю статистику по треку
        statistics[track_id] = {
            "track_id": track_id,
            "total_time_visible_seconds": round(total_time_visible, 2),
            "total_time_without_other_seconds": round(total_time_without_other, 2),
            "num_unique_exercises": num_unique_exercises,
            "exercises": exercise_percentages,
            "total_segments": len(predictions.get(track_id, [])),
            "detection_percentage": track_data.get("detection_percentage", 0),
            "frames_count": len(track_data.get("keypoints_original", [])),
            "start_time": start_time,
            "end_time": end_time,
            "filtered_out_exercises": len(filtered_out_exercises),
            "filtered_out_exercises_list": filtered_out_exercises
        }

    # Преобразуем в DataFrame для удобства
    df_rows = []

    for track_id, stats in statistics.items():
        # Базовая строка
        row = {
            "track_id": track_id,
            "total_time_visible_seconds": stats["total_time_visible_seconds"],
            "total_time_without_other_seconds": stats["total_time_without_other_seconds"],
            "num_unique_exercises": stats["num_unique_exercises"],
            "total_segments": stats["total_segments"],
            "detection_percentage": stats["detection_percentage"],
            "frames_count": stats["frames_count"],
            "start_time": stats["start_time"],
            "end_time": stats["end_time"],
            "time_other_seconds": stats["total_time_visible_seconds"] - stats["total_time_without_other_seconds"],
            "other_percentage": round((stats["total_time_visible_seconds"] - stats["total_time_without_other_seconds"]) /
                                    stats["total_time_visible_seconds"] * 100, 1) if stats["total_time_visible_seconds"] > 0 else 0,
            "filtered_out_exercises": stats["filtered_out_exercises"],
            "filtered_out_exercises_list": ", ".join(stats["filtered_out_exercises_list"]) if stats["filtered_out_exercises_list"] else "нет"
        }

        # Добавляем информацию по каждому упражнению
        for exercise, exercise_stats in stats["exercises"].items():
            row[f"{exercise}_time_seconds"] = exercise_stats["time_seconds"]
            row[f"{exercise}_percentage"] = exercise_stats["percentage"]
            row[f"{exercise}_segments"] = exercise_stats["segments"]

        df_rows.append(row)

    # Создаем DataFrame
    df = pd.DataFrame(df_rows)

    # Заполняем NaN нулями для упражнений, которые были не у всех
    df = df.fillna(0)

    return statistics, df

In [ ]:
def save_statistics_to_csv(tracks, predictions, output_path="track_statistics.csv"):
    """
    Сохраняет статистику по трекам в CSV файл

    Параметры:
    -----------
    tracks : dict
        Словарь с треками
    predictions : dict
        Словарь с предсказаниями
    output_path : str
        Путь для сохранения CSV файла

    Возвращает:
    -----------
    DataFrame : таблица со статистикой
    """

    # Рассчитываем статистику
    statistics, df = calculate_track_statistics(tracks, predictions)

    # Сохраняем в CSV
    df.to_csv(output_path, index=False, encoding='utf-8-sig')

    # Выводим сводную информацию
    print(f"✅ Статистика сохранена в: {output_path}")
    print(f"📊 Всего треков: {len(statistics)}")

    return df, statistics

In [ ]:
def create_statistics_report(statistics, df, output_dir="statistics_report"):
    """
    Создает и сохраняет отдельные графики с анализом статистики

    Параметры:
    -----------
    tracks : dict
        Словарь с треками
    predictions : dict
        Словарь с предсказаниями
    output_dir : str
        Папка для сохранения графиков

    Возвращает:
    -----------
    dict : пути к сохраненным файлам
    """

    # Создаем папку для отчетов
    os.makedirs(output_dir, exist_ok=True)

    # Список для хранения путей к файлам
    saved_files = {}

    # Настройка стиля графиков
    plt.style.use('seaborn-v0_8-darkgrid')
    sns.set_palette("husl")

    # 1. График времени видимости по трекам
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x='track_id', y='total_time_visible_seconds', data=df)
    ax.set_title('Время видимости по трекам', fontsize=14, fontweight='bold')
    ax.set_xlabel('Track ID', fontsize=12)
    ax.set_ylabel('Секунды', fontsize=12)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

    # Добавляем значения на столбцы
    for i, v in enumerate(df['total_time_visible_seconds']):
        ax.text(i, v + 0.5, f'{v:.1f}', ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    time_chart_path = os.path.join(output_dir, "01_time_by_track.png")
    plt.savefig(time_chart_path, dpi=150, bbox_inches='tight')
    plt.close()
    saved_files["time_by_track"] = time_chart_path
    print(f"✅ Сохранен график: {time_chart_path}")

    # 2. График количества уникальных упражнений по трекам
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x='track_id', y='num_unique_exercises', data=df)
    ax.set_title('Количество уникальных упражнений по трекам', fontsize=14, fontweight='bold')
    ax.set_xlabel('Track ID', fontsize=12)
    ax.set_ylabel('Количество упражнений', fontsize=12)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

    for i, v in enumerate(df['num_unique_exercises']):
        ax.text(i, v + 0.1, f'{v}', ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    exercises_chart_path = os.path.join(output_dir, "02_exercises_by_track.png")
    plt.savefig(exercises_chart_path, dpi=150, bbox_inches='tight')
    plt.close()
    saved_files["exercises_by_track"] = exercises_chart_path
    print(f"✅ Сохранен график: {exercises_chart_path}")

    # 3. График процента детекции
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x='track_id', y='detection_percentage', data=df)
    ax.set_title('Процент детекции по трекам', fontsize=14, fontweight='bold')
    ax.set_xlabel('Track ID', fontsize=12)
    ax.set_ylabel('Процент детекции (%)', fontsize=12)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    ax.axhline(y=80, color='r', linestyle='--', alpha=0.5, label='Порог 80%')

    for i, v in enumerate(df['detection_percentage']):
        ax.text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom', fontsize=9)

    plt.legend()
    plt.tight_layout()
    detection_chart_path = os.path.join(output_dir, "03_detection_percentage.png")
    plt.savefig(detection_chart_path, dpi=150, bbox_inches='tight')
    plt.close()
    saved_files["detection_percentage"] = detection_chart_path
    print(f"✅ Сохранен график: {detection_chart_path}")

    # 4. Тепловая карта упражнений по трекам
    # Собираем данные для тепловой карты
    heatmap_data = []
    all_exercises = set()

    for track_id, stats in statistics.items():
        for exercise, ex_stats in stats["exercises"].items():
            heatmap_data.append({
                'track_id': track_id,
                'exercise': exercise,
                'time': ex_stats["time_seconds"]
            })
            all_exercises.add(exercise)

    if heatmap_data and all_exercises:
        heatmap_df = pd.DataFrame(heatmap_data)
        heatmap_pivot = heatmap_df.pivot_table(
            index='track_id',
            columns='exercise',
            values='time',
            aggfunc='sum',
            fill_value=0
        )

        # Сортируем упражнения по общему времени
        exercise_order = heatmap_pivot.sum().sort_values(ascending=False).index
        heatmap_pivot = heatmap_pivot[exercise_order]

        plt.figure(figsize=(12, 8))
        ax = sns.heatmap(
            heatmap_pivot,
            annot=True,
            fmt='.1f',
            cmap='YlOrRd',
            cbar_kws={'label': 'Время (секунды)'},
            linewidths=0.5
        )
        ax.set_title('Тепловая карта: время упражнений по трекам', fontsize=14, fontweight='bold')
        ax.set_xlabel('Упражнения', fontsize=12)
        ax.set_ylabel('Track ID', fontsize=12)
        plt.xticks(rotation=45, ha='right')

        plt.tight_layout()
        heatmap_path = os.path.join(output_dir, "04_exercises_heatmap.png")
        plt.savefig(heatmap_path, dpi=150, bbox_inches='tight')
        plt.close()
        saved_files["exercises_heatmap"] = heatmap_path
        print(f"✅ Сохранен график: {heatmap_path}")

    # 5. Круговая диаграмма распределения упражнений (общая)
    if heatmap_data:
        exercise_totals = heatmap_df.groupby('exercise')['time'].sum()
        exercise_totals = exercise_totals[exercise_totals > 0]

        if len(exercise_totals) > 0:
            plt.figure(figsize=(10, 8))
            colors = plt.cm.Set3(np.linspace(0, 1, len(exercise_totals)))

            wedges, texts, autotexts = plt.pie(
                exercise_totals.values,
                labels=exercise_totals.index,
                autopct='%1.1f%%',
                startangle=90,
                colors=colors,
                pctdistance=0.85
            )

            plt.title('Распределение времени по упражнениям (все треки)', fontsize=14, fontweight='bold')

            # Улучшаем читаемость
            for autotext in autotexts:
                autotext.set_color('black')
                autotext.set_fontsize(10)
                autotext.set_fontweight('bold')

            plt.tight_layout()
            pie_chart_path = os.path.join(output_dir, "05_exercises_distribution.png")
            plt.savefig(pie_chart_path, dpi=150, bbox_inches='tight')
            plt.close()
            saved_files["exercises_distribution"] = pie_chart_path
            print(f"✅ Сохранен график: {pie_chart_path}")

    # 6. Stacked bar chart: время с упражнениями vs "other"
    plt.figure(figsize=(10, 6))

    # Подготовка данных
    track_ids = df['track_id'].astype(str)
    time_without_other = df['total_time_without_other_seconds']
    time_other = df['time_other_seconds']

    p1 = plt.bar(track_ids, time_without_other, label='Время с упражнениями')
    p2 = plt.bar(track_ids, time_other, bottom=time_without_other, label='Время "other"')

    plt.title('Соотношение времени: упражнения vs "other"', fontsize=14, fontweight='bold')
    plt.xlabel('Track ID', fontsize=12)
    plt.ylabel('Секунды', fontsize=12)
    plt.xticks(rotation=45)
    plt.legend()

    # Добавляем общее время сверху
    for i, (total, wo) in enumerate(zip(df['total_time_visible_seconds'], time_without_other)):
        plt.text(i, total + 0.5, f'{total:.1f}', ha='center', va='bottom', fontsize=9)
        # Процент времени с упражнениями
        percentage = (wo / total * 100) if total > 0 else 0
        plt.text(i, wo/2, f'{percentage:.0f}%', ha='center', va='center',
                color='white', fontweight='bold', fontsize=9)

    plt.tight_layout()
    stacked_chart_path = os.path.join(output_dir, "06_time_distribution.png")
    plt.savefig(stacked_chart_path, dpi=150, bbox_inches='tight')
    plt.close()
    saved_files["time_distribution"] = stacked_chart_path
    print(f"✅ Сохранен график: {stacked_chart_path}")

    # 7. График количества сегментов по трекам
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x='track_id', y='total_segments', data=df)
    ax.set_title('Количество сегментов по трекам', fontsize=14, fontweight='bold')
    ax.set_xlabel('Track ID', fontsize=12)
    ax.set_ylabel('Количество сегментов', fontsize=12)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

    for i, v in enumerate(df['total_segments']):
        ax.text(i, v + 0.1, f'{v}', ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    segments_chart_path = os.path.join(output_dir, "07_segments_by_track.png")
    plt.savefig(segments_chart_path, dpi=150, bbox_inches='tight')
    plt.close()
    saved_files["segments_by_track"] = segments_chart_path
    print(f"✅ Сохранен график: {segments_chart_path}")

    # 8. Box plot времени видимости
    plt.figure(figsize=(8, 6))
    sns.boxplot(y=df['total_time_visible_seconds'])
    plt.title('Распределение времени видимости треков', fontsize=14, fontweight='bold')
    plt.ylabel('Секунды', fontsize=12)

    # Добавляем среднее значение
    mean_time = df['total_time_visible_seconds'].mean()
    plt.axhline(y=mean_time, color='r', linestyle='--', alpha=0.7,
                label=f'Среднее: {mean_time:.1f} сек')

    # Добавляем точки для каждого трека
    for i, time in enumerate(df['total_time_visible_seconds']):
        plt.scatter(0, time, alpha=0.6, s=50)

    plt.legend()
    plt.tight_layout()
    boxplot_path = os.path.join(output_dir, "08_time_distribution_boxplot.png")
    plt.savefig(boxplot_path, dpi=150, bbox_inches='tight')
    plt.close()
    saved_files["time_boxplot"] = boxplot_path
    print(f"✅ Сохранен график: {boxplot_path}")

    return saved_files, df, statistics

In [ ]:
df, statistics = save_statistics_to_csv(tracks, predictions, "track_statistics.csv")

saved_files, _, _ = create_statistics_report(
    statistics, df,
    output_dir="statistics_report")

🔍 Фильтрация треков: минимум 3 детекции (сегмента)
  Сохранен трек 1: 20 сегментов
  Сохранен трек 5: 6 сегментов
  Пропущен трек 7: только 0 сегментов
  Пропущен трек 9: только 1 сегментов
  Пропущен трек 20: только 0 сегментов
  Пропущен трек 21: только 1 сегментов
📊 После фильтрации треков: 2 из 6 треков
  Трек 1: упражнение 'leg raises' - 7 сегментов
  Трек 1: упражнение 'tricep Pushdown' - 4 сегментов
  Трек 1: упражнение 'tricep dips' - 4 сегментов
  Трек 1: пропущены упражнения ['chest fly machine', 'lateral raise', 'leg extension', 'barbell biceps curl', 'other'] (≤2 сегментов)
  Трек 5: упражнение 'pull Up' - 4 сегментов
  Трек 5: пропущены упражнения ['other'] (≤2 сегментов)
✅ Статистика сохранена в: track_statistics.csv
📊 Всего треков: 2
✅ Сохранен график: statistics_report/01_time_by_track.png


/tmp/ipython-input-3467851233.py:35: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
/tmp/ipython-input-3467851233.py:54: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45)


✅ Сохранен график: statistics_report/02_exercises_by_track.png
✅ Сохранен график: statistics_report/03_detection_percentage.png


/tmp/ipython-input-3467851233.py:72: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45)


✅ Сохранен график: statistics_report/04_exercises_heatmap.png
✅ Сохранен график: statistics_report/05_exercises_distribution.png
✅ Сохранен график: statistics_report/06_time_distribution.png


/tmp/ipython-input-3467851233.py:206: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45)


✅ Сохранен график: statistics_report/07_segments_by_track.png
✅ Сохранен график: statistics_report/08_time_distribution_boxplot.png


### Визуализация

In [ ]:
def visualize_with_exercises(video_path, tracks, predictions, output_path="tracking_with_exercises.mp4"):
    """
    Визуализация: скелеты разными цветами + подписи упражнений
    """
    import cv2
    import numpy as np
    from tqdm import tqdm
    import os

    # Открываем видео
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Ошибка: не удалось открыть видео {video_path}")
        return

    # Параметры видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Создание визуализации: {width}x{height}, FPS: {fps:.1f}")

    # Создаем VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Цвета для разных треков (BGR формат в OpenCV)
    colors = [
        (0, 255, 0),    # зеленый
        (255, 0, 0),    # синий
        (0, 0, 255),    # красный
        (255, 255, 0),  # голубой
        (255, 0, 255),  # розовый
        (0, 255, 255),  # желтый
        (255, 255, 255),# белый
    ]

    # Создаем карту кадр→треки
    frame_map = {}
    for track_id, track_data in tracks.items():
        frame_indices = track_data.get("frame_indices", [])
        keypoints = track_data.get("keypoints_original", [])

        for i, frame_idx in enumerate(frame_indices):
            if i < len(keypoints):
                if frame_idx not in frame_map:
                    frame_map[frame_idx] = []
                frame_map[frame_idx].append((track_id, keypoints[i]))

    # Обрабатываем видео
    print("\nОбработка видео...")

    for frame_idx in tqdm(range(total_frames), desc="Кадры"):
        ret, frame = cap.read()
        if not ret:
            break

        current_time = frame_idx / fps if fps > 0 else 0

        # Рисуем треки в текущем кадре
        if frame_idx in frame_map:
            for track_id, keypoints in frame_map[frame_idx]:
                # Цвет для трека
                color = colors[track_id % len(colors)]

                # Рисуем скелет (упрощенный)
                if len(keypoints) == 51:
                    kp = keypoints.reshape(17, 3)
                else:
                    kp = keypoints

                # Соединения
                connections = [
                    (0, 1), (0, 2), (1, 3), (2, 4),
                    (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),
                    (11, 12), (11, 13), (13, 15), (12, 14), (14, 16),
                    (5, 11), (6, 12)
                ]

                # Рисуем линии
                for start_idx, end_idx in connections:
                    if start_idx < len(kp) and end_idx < len(kp):
                        x1, y1, c1 = kp[start_idx]
                        x2, y2, c2 = kp[end_idx]

                        if c1 > 0.3 and c2 > 0.3:
                            if 0 <= x1 < width and 0 <= y1 < height and 0 <= x2 < width and 0 <= y2 < height:
                                cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

                # Рисуем точки
                for i, (x, y, conf) in enumerate(kp):
                    if conf > 0.3 and 0 <= x < width and 0 <= y < height:
                        cv2.circle(frame, (int(x), int(y)), 4, color, -1)

                # Подпись с упражнением
                exercise_text = f"ID:{track_id}"
                if predictions and track_id in predictions:
                    for segment in predictions[track_id]:
                        if segment["start_time"] <= current_time <= segment["end_time"]:
                            exercise_text = f"ID:{track_id} - {segment['predicted_class']}"
                            break

                # Позиция для текста
                visible = kp[kp[:, 2] > 0.3]
                if len(visible) > 0:
                    text_x = int(np.mean(visible[:, 0]))
                    text_y = int(np.min(visible[:, 1])) - 20

                    text_x = max(10, min(text_x, width - 200))
                    text_y = max(30, text_y)

                    # Фон
                    (w, h), _ = cv2.getTextSize(exercise_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                    cv2.rectangle(frame,
                                (text_x - 5, text_y - h - 5),
                                (text_x + w + 5, text_y + 5),
                                (0, 0, 0), -1)

                    # Текст
                    cv2.putText(frame, exercise_text,
                              (text_x, text_y),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Время и номер кадра
        time_str = f"{int(current_time//60):02d}:{current_time%60:05.2f}"
        cv2.putText(frame, f"Time: {time_str} | Frame: {frame_idx}/{total_frames}",
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        # Записываем кадр
        out.write(frame)

    # Освобождаем ресурсы
    cap.release()
    out.release()

    print(f"\n✅ Визуализация сохранена: {output_path}")

    return output_path

In [ ]:
video_output = visualize_with_exercises(
    video_path="/content/test_final.mp4",
    tracks=tracks,
    predictions=predictions,
    output_path="exercise_tracking_final.mp4"
)

Создание визуализации: 1920x1080, FPS: 30.0

Обработка видео...


Кадры: 100%|██████████| 1020/1020 [00:25<00:00, 39.49it/s]


✅ Визуализация сохранена: exercise_tracking_final.mp4


### Тест пайплайна

In [ ]:
!unzip -q /content/workout_pipeline.zip

In [ ]:
!pip install -r /content/workout_pipeline/requirements.txt

In [ ]:
!python /content/workout_pipeline/main.py --video_path /content/test_final.mp4 --model_path /content/best_pose_lstm_new.pth --visualize --stats

Загрузка модели из /content/best_pose_lstm_new.pth...
Модель загружена на cuda
Трекинг видео: /content/test_final.mp4

Обработка человека ID 1
  Сегментов: 16

Обработка человека ID 5
  Сегментов: 2

Обработка человека ID 7
  Сегментов: 0

Обработка человека ID 8
  Сегментов: 0
✅ Tracks сохранены: output/2025-12-23_05-35-16/tracks.pkl
✅ Predictions сохранены: output/2025-12-23_05-35-16/predictions.pkl

Создание визуализации...
Создание визуализации: 1920x1080, FPS: 30.0

Обработка видео...
Кадры: 100% 1020/1020 [00:21<00:00, 47.07it/s]

 Визуализация сохранена: output/2025-12-23_05-35-16/exercise_tracking_output.mp4
✅ Видео сохранено: output/2025-12-23_05-35-16/exercise_tracking_output.mp4

Расчет статистик и создание графиков...
🔍 Фильтрация треков: минимум 3 детекции (сегмента)
  Сохранен трек 1: 16 сегментов
  Пропущен трек 5: только 2 сегментов
  Пропущен трек 7: только 0 сегментов
  Пропущен трек 8: только 0 сегментов
📊 После фильтрации треков: 1 из 4 треков
  Трек 1: упражнение '